In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy import stats
import pickle
import glob

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# import the relevant Keras modules
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, LSTM, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

dataset_path = '/kaggle/input/battery-and-heating-data-in-real-driving-cycles'

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install openpyxl

# Load dataset

In [ ]:
all_files = glob.glob(dataset_path + "/TripB01.csv")
df = (pd.read_csv(dataset_path + "/TripB01.csv", sep=';', encoding="ISO-8859-1"))

In [ ]:
df.head()

In [ ]:
plt.plot(df['Time [s]'], df['Battery Voltage [V]'], label='Voltage in V')
plt.xlabel('Time [s]')
plt.ylabel('Battery Voltage [V]')
plt.legend()
plt.show()

plt.plot(df['Time [s]'], df['Battery Current [A]'], label='Current in A)')
plt.xlabel('Time [s]')
plt.ylabel('Battery Current [A]')
plt.legend()
plt.show()

plt.plot(df['Time [s]'], df['SoC [%]'], label='State of charge in %')
plt.xlabel('Time [s]')
plt.ylabel('SoC [%]')
plt.legend()
plt.show()

In [ ]:
df = df.filter(['Battery Voltage [V]', 'Battery Current [A]', 'SoC [%]'])

# Convert series to supervised problem

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

reframed = series_to_supervised(df, 1, 1)
reframed.head()

# Prepare data for training

In [ ]:
# Train-test split
values = reframed.values
train, test = train_test_split(values)

# Split into input and output
X_train, y_train = train[:, [0, 1, 2]], train[:, [3, 4, 5]]
X_test, y_test = test[:, [0, 1, 2]], test[:, [3, 4, 5]]

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X_train = scaler1.fit_transform(X_train)
y_train = scaler2.fit_transform(y_train)

X_test = scaler1.transform(X_test)
y_test = scaler2.transform(y_test)


# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Define and compile model

In [ ]:
model = Sequential()
model.add(LSTM(50, activation = 'tanh', recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(LSTM(256, activation = 'tanh', recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, return_sequences = True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(X_train.shape[2]))

model.compile(loss = 'mae', optimizer = Adam(lr = 1e-3))

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        min_delta=1e-4,
        patience=5,
        verbose=1
    ),
    ModelCheckpoint(
        filepath="weights.h5", 
        monitor="val_loss", 
        mode='min', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), 
          verbose=2, shuffle=False, callbacks=callbacks)

model.load_weights('weights.h5')
# model.save('model.pb')

# Plot predictions vs actual data

In [ ]:
# yplot = scaler2.inverse_transform(y_test)
# xplot = model.predict(X_test)

# print(model.predict(X_train).shape, y_test.shape)

# plt.rcParams['agg.path.chunksize'] = 10000
matplotlib.rcParams.update({'font.size': 22})

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
plt.plot(scaler2.inverse_transform(y_test[:100])[:, [0]], label = "actual", linewidth=3.0, c='b')
plt.plot(scaler1.inverse_transform(model.predict(X_test[:100]))[:, [0]], label = "predicted", c='r')
plt.xlabel('Time [s]')
plt.ylabel('Battery Voltage [V]')
plt.legend()
plt.show()

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
plt.plot(scaler2.inverse_transform(y_test[:100])[:, [1]], label = "actual", linewidth=3.0, c='b')
plt.plot(scaler1.inverse_transform(model.predict(X_test[:100]))[:, [1]], label = "predicted", c='r')
plt.xlabel('Time [s]')
plt.ylabel('Battery Current [A]')
plt.legend()
plt.show()

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
plt.plot(scaler2.inverse_transform(y_test[:100])[:, [2]], label = "actual", linewidth=3.0, c='b')
plt.plot(scaler1.inverse_transform(model.predict(X_test[:100]))[:, [2]], label = "predicted", c='r')
plt.xlabel('Time [s]')
plt.ylabel('SoC %')
plt.legend()
plt.show()